In [1]:
import serial
from anatomical_hinge_nagillimi.anatomical_hinge import AnatomicalHinge
from anatomical_hinge_nagillimi.data import Data
from anatomical_hinge_nagillimi.result.joint_result import HingeJointResult

hinge = AnatomicalHinge()
ser = serial.Serial("COM8", 38400)

print("Connected to port: ", ser.portstr)

print("Calibrating...")
ser.write(bytes(b' '))
ser.readline()

print("Incoming data config")
print(ser.readline())
serialData = ser.readline()

print("Length of packet:", len(serialData))
print("Packet type:", type(serialData), type(serialData[0]))
print("First timestamp:", int(serialData[0]), "\n")

Connected to port:  COM8
Calibrating...
Incoming data config
b'CONFIG: [ts, pot, ax1, ay1, az1, gx1, gy1, gz1, ax2, ay2, az2, gx2, gy2, gz2]\r\n'
Length of packet: 71
Packet type: <class 'bytes'> <class 'int'>
First timestamp: 49 



In [2]:
# env constants
max_raw_range = 16384 * 2
g_per_raw_range = 2 / max_raw_range
dps_per_raw_range = 250 / max_raw_range

In [3]:
import math

# Define the read data function
def readData():
    byteData = ser.readline().split(b"\x09")
    ts = int(byteData[0]) / 1000
    a1 = [
        float(byteData[1]) * g_per_raw_range,
        float(byteData[2]) * g_per_raw_range,
        float(byteData[3]) * g_per_raw_range
    ]
    g1 = [
        math.radians(float(byteData[4]) * dps_per_raw_range),
        math.radians(float(byteData[5]) * dps_per_raw_range),
        math.radians(float(byteData[6]) * dps_per_raw_range)
    ]
    a2 = [
        float(byteData[7]) * g_per_raw_range,
        float(byteData[8]) * g_per_raw_range,
        float(byteData[9]) * g_per_raw_range
    ]
    g2 = [
        math.radians(float(byteData[10]) * dps_per_raw_range),
        math.radians(float(byteData[11]) * dps_per_raw_range),
        math.radians(float(byteData[12]) * dps_per_raw_range)
    ]
    return Data(int(ts), a1, g1, a2, g2)

In [6]:
# Calibrate
byteData = ser.readline().split(b"\x09")
while len(byteData) != 0:
    data = readData()
    hinge.update(data)
    print(hinge.axisCalibration.derivSumOfSquares)
    if hinge.status == HingeJointResult.STREAMING: break

TypeError: Object of type AnatomicalHinge is not JSON serializable

In [ ]:
for i in range(1000):
    print(hinge.update(readData()))

In [ ]:
# def animate(line, ys, x_len):
#     data = readData()
#     # print(data)
#     ys.append(data)
#     ys = ys[-x_len:]
#     line.set_ydata(ys)
#     return line,

In [ ]:
# from time import sleep
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation

# # Create figure for plotting
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)

# x_len = 100
# xs = list(range(0, x_len))
# ys = [0] * x_len
# ax.set_ylim(-300, 300)

# line, = ax.plot(xs, ys)

# # Format plot
# plt.xticks(rotation=45, ha='right')
# plt.subplots_adjust(bottom=0.30)

# # Set up plot to call animate() function periodically
# ani = animation.FuncAnimation(fig, animate, fargs=(line, ys, x_len), interval=50, blit=True)
# plt.show()

# sleep(10000)